De

In [65]:
import pandas as pd

In [66]:
white_wine = pd.read_csv(".\DataSet\winequality-white.csv", sep=";",quotechar='"')

red_wine = pd.read_csv(".\DataSet\winequality-red.csv", sep=";",quotechar='"')


In [67]:
white_wine.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


In [68]:
white_wine.describe()


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000
mean,6.854788,0.278241,0.334192,6.391415,0.045772,35.308085,138.360657,0.994027,3.188267,0.489847,10.514267,5.877909
std,0.843868,0.100795,0.121020,5.072058,0.021848,17.007137,42.498065,0.002991,0.151001,0.114126,1.230621,0.885639
min,3.800000,0.080000,0.000000,0.600000,0.009000,2.000000,9.000000,0.987110,2.720000,0.220000,8.000000,3.000000
25%,6.300000,0.210000,0.270000,1.700000,0.036000,23.000000,108.000000,0.991723,3.090000,0.410000,9.500000,5.000000
50%,6.800000,0.260000,0.320000,5.200000,0.043000,34.000000,134.000000,0.993740,3.180000,0.470000,10.400000,6.000000
75%,7.300000,0.320000,0.390000,9.900000,0.050000,46.000000,167.000000,0.996100,3.280000,0.550000,11.400000,6.000000
max,14.200000,1.100000,1.660000,65.800000,0.346000,289.000000,440.000000,1.038980,3.820000,1.080000,14.200000,9.000000


In [69]:
white_wine.corr()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
fixed acidity,1.000000,-0.022697,0.289181,0.089021,0.023086,-0.049396,0.091070,0.265331,-0.425858,-0.017143,-0.120881,-0.113663
volatile acidity,-0.022697,1.000000,-0.149472,0.064286,0.070512,-0.097012,0.089261,0.027114,-0.031915,-0.035728,0.067718,-0.194723
citric acid,0.289181,-0.149472,1.000000,0.094212,0.114364,0.094077,0.121131,0.149503,-0.163748,0.062331,-0.075729,-0.009209
residual sugar,0.089021,0.064286,0.094212,1.000000,0.088685,0.299098,0.401439,0.838966,-0.194133,-0.026664,-0.450631,-0.097577
chlorides,0.023086,0.070512,0.114364,0.088685,1.000000,0.101392,0.198910,0.257211,-0.090439,0.016763,-0.360189,-0.209934
free sulfur dioxide,-0.049396,-0.097012,0.094077,0.299098,0.101392,1.000000,0.615501,0.294210,-0.000618,0.059217,-0.250104,0.008158
total sulfur dioxide,0.091070,0.089261,0.121131,0.401439,0.198910,0.615501,1.000000,0.529881,0.002321,0.134562,-0.448892,-0.174737
density,0.265331,0.027114,0.149503,0.838966,0.257211,0.294210,0.529881,1.000000,-0.093591,0.074493,-0.780138,-0.307123
pH,-0.425858,-0.031915,-0.163748,-0.194133,-0.090439,-0.000618,0.002321,-0.093591,1.000000,0.155951,0.121432,0.099427
sulphates,-0.017143,-0.035728,0.062331,-0.026664,0.016763,0.059217,0.134562,0.074493,0.155951,1.000000,-0.017433,0.053678


In [70]:
%matplotlib
import matplotlib.pyplot as plot

Using matplotlib backend: Qt5Agg


In [71]:
plot.rcParams.update({'font.size': 8, 'figure.figsize': (10, 8)}) # set font and plot size to be larger


In [72]:
img = white_wine.hist(bins=50)

plot.show()

In [73]:
import seaborn as sns

In [74]:
sns.set(font_scale=1.1) # incrementa il font
# colour = "tab20"
# colour = "tab20c"
# colour = "Pastel2"
# colour = "Paired" # <- print
colour = "Spectral"
# colour ="nipy_spectral_r" # <- Print e plot

In [75]:
sns.heatmap(white_wine.corr(), annot=True, cmap=colour, linewidths=0.5)
plot.show()

In [76]:
import os

def saveImage(dataFramePlot, imageName: str, path='.\\Exports\\', extension=".png"):
    if not os.path.isfile(f"{path}{imageName}{extension}"):
        fig = dataFramePlot.get_figure()
        fig.savefig(f"{path}{imageName}{extension}")

In [77]:
val = "alcohol"
imgPlot = white_wine[val].plot(kind="box", figsize=(5,10), grid=True)
saveImage(imgPlot, f"boxPlot_{val}")


In [78]:
val = "volatile acidity"
imgPlot = white_wine[val].plot(kind="box", figsize=(5,10), grid=True)
saveImage(imgPlot, f"boxPlot_{val}")


In [79]:
val = "citric acid"
imgPlot = white_wine[val].plot(kind="box", figsize=(5,10), grid=True)
saveImage(imgPlot, f"boxPlot_{val}")


In [80]:
val = "chlorides"
imgPlot = white_wine[val].plot(kind="box", figsize=(5,10), grid=True)
saveImage(imgPlot, f"boxPlot_{val}")


In [81]:
val = "residual sugar"
imgPlot = white_wine[val].plot(kind="box", figsize=(5,10), grid=True)
saveImage(imgPlot, f"boxPlot_{val}")


In [82]:
val = "fixed acidity"
imgPlot = white_wine[val].plot(kind="box", figsize=(5,10), grid=True)
saveImage(imgPlot, f"boxPlot_{val}")


In [83]:
val = "free sulfur dioxide"
imgPlot = white_wine[val].plot(kind="box", figsize=(5,10), grid=True)
saveImage(imgPlot, f"boxPlot_{val}")


In [84]:
val = "total sulfur dioxide"
imgPlot = white_wine[val].plot(kind="box", figsize=(5,10), grid=True)
saveImage(imgPlot, f"boxPlot_{val}")


In [85]:
val = "density"
imgPlot = white_wine[val].plot(kind="box", figsize=(5,10), grid=True)
saveImage(imgPlot, f"boxPlot_{val}")


In [86]:
val = "pH"
imgPlot = white_wine[val].plot(kind="box", figsize=(5,10), grid=True)
saveImage(imgPlot, f"boxPlot_{val}")


In [87]:
val = "sulphates"
imgPlot = white_wine[val].plot(kind="box", figsize=(5,10), grid=True)
saveImage(imgPlot, f"boxPlot_{val}")


In [88]:
val = "quality"
imgPlot = white_wine[val].plot(kind="box", figsize=(5,10), grid=True)
saveImage(imgPlot, f"boxPlot_{val}")


In [89]:
# # prendo le correlazioni escludendo la qualità (valore da predire)
# correlations = white_wine.corr()['quality'].drop('quality')
# print(correlations)

In [90]:
# def get_features(correlation_threshold):
#     abs_corrs = correlations.abs()
#     high_correlations = abs_corrs[abs_corrs > correlation_threshold].index.values.tolist()
#     return high_correlations

In [91]:
# # taking features with correlation more than 0.05 as input x and quality as target variable y 
# features = get_features(0.05) 
# print(features) 
# relevantFeautresDF = white_wine[features] 
# qualitySeries = white_wine['quality']

In [92]:
# relevantFeautresDF.plot(kind="scatter", figsize=(5,10), grid=True)

In [93]:
from sklearn import preprocessing 
from sklearn.preprocessing import LabelEncoder

In [94]:
columnName = "quality"

white_wine[columnName].describe()

count    4898.000000
mean        5.877909
std         0.885639
min         3.000000
25%         5.000000
50%         6.000000
75%         6.000000
max         9.000000
Name: quality, dtype: float64

In [95]:
labelEncoder = preprocessing.LabelEncoder()
labelEncoder.fit(white_wine[columnName].astype(float))
list(labelEncoder.classes_)


[3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0]

In [96]:
columnName = "quality"
# very good = 1 -> 6:1,7:1,8:1,9:1
# not good = 0 -> 3:0,4:0,5:0
white_wine[columnName].replace(to_replace={6:1,7:1,8:1,9:1, 3:0,4:0,5:0}, inplace=True)
white_wine.head(10)


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,1
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,1
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,1
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,1
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,1
5,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,1
6,6.2,0.32,0.16,7.0,0.045,30.0,136.0,0.9949,3.18,0.47,9.6,1
7,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,1
8,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,1
9,8.1,0.22,0.43,1.5,0.044,28.0,129.0,0.9938,3.22,0.45,11.0,1


In [97]:
X = white_wine.drop([columnName], axis=1) # predittori
Y = white_wine[columnName].astype(int) # da predire
white_wine.plot()


In [98]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
dtypes: float64(11)
memory usage: 421.0 KB


In [99]:
X.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9


In [100]:
Y.count()

4898

In [101]:
Y.head()


0    1
1    1
2    1
3    1
4    1
Name: quality, dtype: int32

In [102]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=5)

In [103]:
print(f"X train {x_train.shape}\nY train {y_train.shape}\n")

print(f"X test {x_test.shape}\nY test {x_test.shape}")


X train (3673, 11)
Y train (3673,)

X test (1225, 11)
Y test (1225, 11)


In [104]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

In [105]:
# def print_metrics(y_test, y_pred):
#     print('1. Accuracy: {}'.format(round(accuracy_score(y_test, y_pred))))
#     # TODO AGGIUSTAMENTI PARAMETRI
#     print('2. Precision: {}'.format(round(precision_score(y_test, y_pred, average="micro"))))
#     print('3. Recall: {}'.format(round(recall_score(y_test, y_pred, average="micro"))))
#     print('4. F1 score: {}'.format(round(f1_score(y_test, y_pred, average="micro"))))
#     print('5. ROC AUC: {}'.format(round(roc_auc_score(y_test, y_pred, average="weighted", multi_class="ovr"))))
#     # print('\n++ Confusion Matrix ++\n{}'.format(pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted'])))



def print_metrics(y_test, y_pred):
    print('1. Accuracy: {}'.format(round(accuracy_score(y_test, y_pred), 2)))
    print('2. Precision: {}'.format(round(precision_score(y_test, y_pred), 2)))
    print('3. Recall: {}'.format(round(recall_score(y_test, y_pred), 2)))
    print('4. F1 score: {}'.format(round(f1_score(y_test, y_pred), 2)))
    print('5. ROC AUC: {}'.format(round(roc_auc_score(y_test, y_pred), 2)))
    print('\n++ Confusion Matrix ++\n{}'.format(pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted'])))

In [106]:
# Let's create a preprocessor for data normalization
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

# For first, we will process train set
x_train_scaled = scaler.fit_transform(x_train)

# Then, we will do the same for test set
x_test_scaled = scaler.transform(x_test)

In [107]:
# 1. Model trainig
model = SVC().fit(x_train_scaled, y_train)

# 2. Prediction
y_pred = model.predict(x_test_scaled)

In [108]:
# Let's print some metrics
print_metrics(y_test, y_pred)

1. Accuracy: 0.77
2. Precision: 0.8
3. Recall: 0.88
4. F1 score: 0.84
5. ROC AUC: 0.72

++ Confusion Matrix ++
Predicted    0    1
Actual             
0          224  180
1          100  721


In [109]:
# from sklearn.metrics import plot_confusion_matrix
# plot_confusion_matrix(model, x_test, y_pred)
# # plot_confusion_matrix(model, x_test, y_test)  # doctest: +SKIP
# plt.show()  # doctest: +SKIP

In [110]:
# from sklearn.metrics import plot_confusion_matrix

# titles_options = [("Confusion matrix, without normalization", None),
#                   ("Normalized confusion matrix", 'true')]
                  
# for title, normalize in titles_options:
#     disp = plot_confusion_matrix(model, x_test, y_test,
#                                  display_labels=columnName,
#                                  cmap=plot.cm.Blues,
#                                  normalize=normalize)
#     disp.ax_.set_title(title)

#     print(title)
#     print(disp.confusion_matrix)

# plt.show()

In [112]:
sns.heatmap(white_wine.corr(), annot=True, cmap=colour, linewidths=0.5)
plot.show()

In [53]:
%conda install -c conda-forge imbalanced-learn


Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.

Note: you may need to restart the kernel to use updated packages.

PackagesNotFoundError: The following packages are not available from current channels:

  - imblearn

Current channels:

  - https://conda.anaconda.org/conda-forge/win-64
  - https://conda.anaconda.org/conda-forge/noarch
  - https://repo.anaconda.com/pkgs/main/win-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/win-64
  - https://repo.anaconda.com/pkgs/r/noarch
  - https://repo.anaconda.com/pkgs/msys2/win-64
  - https://repo.anaconda.com/pkgs/msys2/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.




In [113]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

In [115]:
# white_wine.loc[white_wine["quality"] == 0] # 1640 samples
# white_wine.loc[white_wine["quality"] == 1] # 3258 samplse


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,1
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,1
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,1
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,1
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,1
...,...,...,...,...,...,...,...,...,...,...,...,...
4891,5.7,0.21,0.32,0.9,0.038,38.0,121.0,0.99074,3.24,0.46,10.6,1
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,1
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,1
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,1


In [118]:
# For train set only !!!
rus = RandomUnderSampler(sampling_strategy="majority") 
x_rus, y_rus = rus.fit_sample(x_train_scaled, y_train)

AttributeError: 'RandomUnderSampler' object has no attribute '_validate_data'